Вас пригласили на работу в коммерческую компанию, занимающуюся разработкой автоматизированных торговых агентов. Одной из первых ваших задач будет подготовка данных для дальнейшей обработки и построения модели. Пообщавшись с коллегами, вы узнали, что вам предстоит работать с несколькими типами активов: акциями из списка SnP500 и криптовалютами (BTC, ETH, SOL, XRP). Вам планируют поручить краткосрочную и среднесрочную торговлю.


Вам предлагается на основе предоставленной информации:


1. Создать git-репозиторий, где будет храниться исходный код вашего проекта. Если вы используете приватный репозиторий – дайте преподавателям курса доступ к нему, для возможности проверки ДЗ.
2. Добавить файл лицензии, который отражает ваш взгляд на конфиденциальность информации, которую вы подготовите в рамках данного курса.
3. Создать код на Python, который загрузит на ваш локальный компьютер данные о котировках ценных бумаг из списка SnP500 и котировки криптовалют (BTC, ETH, SOL, XRP).
4. Поскольку вам предстоит много работать с ними в дальнейшем, подготовьте автоматическое отображение графиков текущей ситуации.
5. Проверьте нет ли в данных пропусков или ошибок. Проанализируйте выбросы. Оцените, на самом ли деле это выбросы или реальные данные, с которыми предстоит работать.

In [ ]:
# Initialization cell

!pip install yfinance -qq
import yfinance as yf
import pandas_datareader as web
import pandas as pd
import os
import plotly.graph_objects as go


# Create Data directory if it doesn't exist
dir_stocks = os.path.join('data', 'stock')
if not os.path.exists(dir_stocks):
    os.makedirs(dir_stocks)
dir_crypto = os.path.join('data', 'crypto')
if not os.path.exists(dir_crypto):
    os.makedirs(dir_crypto)

In [ ]:
# Data download to subfolders cell

tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].str.replace('.','-', regex=True).to_list()[:10]
data = yf.download(tickers=tickers,group_by='Ticker',multi_level_index=False,progress=False)
data_per_ticker = {}
for ticker in tickers:
    df = data[ticker].dropna()
    df.to_csv(os.path.join(dir_stocks, ticker + '.csv'))


# Download crypto data
cryptos = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'XRP-USD']
for crypto in cryptos:
    data = yf.download(crypto, multi_level_index=False,progress=False).dropna()
    data.to_csv(os.path.join(dir_crypto, crypto.split("-")[0] + '.csv'))


In [ ]:
# Functions definition cell

def load_data(ticker: str, is_crypto: bool = False):
    df: pd.DataFrame = None
    path = dir_stocks
    if is_crypto:
        path = dir_crypto
    path = os.path.join(path, ticker + '.csv')
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=['Date'])
    else:
        print(f'No file for {ticker} at {path}')
    return df

def plot(df, time_start = None, time_end = None):
    if time_start is None:
        time_start = min(df['Date'])
    if time_end is None:
        time_end = max(df['Date'])
    df = df[(df['Date'] >= start_time) & (df['Date'] <= end_time)]
    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
    fig.show()        

In [ ]:
df = load_data('A')
time_start = pd.to_datetime('2000-01-01')
time_end   = pd.to_datetime('2000-01-05')
plot(df, time_start, time_end)